In [ ]:
#coding: utf-8
import numpy as np
import pandas as pd
import re
import jieba
import os
import time
import random
import jieba  #处理中文
import sklearn #分类器
from sklearn.naive_bayes import MultinomialNB  #也可以换成伯努利或高斯的贝叶斯试试看
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import time
import gensim
import jieba.analyse
import time

In [ ]:
from gensim.models import CoherenceModel
from gensim import models
def tw_lda_get_tfidf(text,gamma=1.0):
    #1.用正文+标题作为dictionary，过滤极端值
    #--------------下面这里修改
    content_and_title=[i.split(" ") for i in text["微博正文(无标题切词后)"]]
    dictionary = gensim.corpora.Dictionary(content_and_title)
    #-------------下面这里修改
    dictionary.filter_extremes(no_below=15,no_above=0.8, keep_n=100000)
    
    #2.计算标题的tfidf
    #--------------下面这里修改
    processed_docs_title=[i.split(" ") for i in text["标题(切词后)"]]
    bow_corpus_title = [dictionary.doc2bow(doc) for doc in processed_docs_title]
    tfidf_title = models.TfidfModel(bow_corpus_title,normalize=False)
    corpus_tfidf_title = tfidf_title[bow_corpus_title]
#     print("corpus_tfidf_title",corpus_tfidf_title)
    
    #3.计算正文+标题的tfidf
    #--------------下面这里修改
    processed_docs_content=[i.split(" ") for i in text["微博正文(无标题切词后)"]]
    bow_corpus_content = [dictionary.doc2bow(doc) for doc in processed_docs_content]
    tfidf_content = models.TfidfModel(bow_corpus_content,normalize=False)
    corpus_tfidf_content = tfidf_content[bow_corpus_content]
#     print("corpus_tfidf_content",corpus_tfidf_content)
    
    #4.把标题和正文tfidf结合在一起
    new_tfidf=[]
    for i in range(len(corpus_tfidf_content)):#corpus_tfidf2[i]
        dict_2={one:two for one,two in corpus_tfidf_content[i]}
        dict_1={one:two for one,two in corpus_tfidf_title[i]}
        for j in dict_1.keys():
            if j in dict_2.keys():
                dict_2[j]=(1-gamma)*dict_2[j]+gamma*dict_1[j]
        new_tfidf_part=[(one,two) for one,two in sorted(dict_2.items(), key=lambda d: d[0],reverse=False)]
#         print("new_tfidf_part",new_tfidf_part)
#        new_tfidf.append(new_tfidf_part)
#        normed=gensim.models.tfidfmodel.smartirs_normalize(new_tfidf_part,"c")  #-------------这里改了把这个取消归一化了
#        new_tfidf.append(normed)
        new_tfidf.append(new_tfidf_part)
    
    #5.返回tfidf
    return new_tfidf,dictionary


# lda的模型
from gensim.models import CoherenceModel
from gensim import models

def lda_and_coherence_score(processed_docs,num_topics,methods,corpus_tfidf_tw_lda=False,dictionary_tw_lda=False):
    
    if methods=="tw_lda":
        dictionary=dictionary_tw_lda
        corpus=corpus_tfidf_tw_lda
    else:
        dictionary = gensim.corpora.Dictionary(processed_docs)
        dictionary.filter_extremes(no_below=15, no_above=0.8, keep_n=100000)
        bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
        if methods=="lda_normal":
            corpus=bow_corpus
        elif methods=="lda_tfidf":
            tfidf = models.TfidfModel(bow_corpus,normalize=False)#-----------------这里改了，改成false
            corpus_tfidf = tfidf[bow_corpus]
            corpus=corpus_tfidf
    model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary,
                                       passes=20,iterations=5000
                                       #,eval_every=1
                                      )
    coherence = CoherenceModel(model=model, texts=processed_docs,dictionary=dictionary, coherence='c_v')
    score = coherence.get_coherence()
    logper=model.log_perplexity(corpus)
    return model,score,logper


#----------增加一步，算5次平均【def名中的10没改，但实际是算5次】
def cs_bar_10(c,num_topics,methods,corpus_tfidf_tw_lda=False,dictionary_tw_lda=False):
    cs_list=[]
    logper_list=[]
    if methods=="tw_lda":
        for i in range(5):
            lda,cs,logper=lda_and_coherence_score(c,num_topics,methods,corpus_tfidf_tw_lda=corpus_tfidf_tw_lda,dictionary_tw_lda=dictionary_tw_lda)
            cs_list.append(cs)
            logper_list.append(logper)
    else:
        for i in range(5):
            lda,cs,logper=lda_and_coherence_score(c,num_topics,methods)
            cs_list.append(cs)
            logper_list.append(logper)
#     cs_bar_10=sum(cs_list)/(len(cs_list)*1.0)               #----------------这里也改了
#     logper_bar_10=sum(logper_list)/(len(logper_list)*1.0)   #----------------这里也改了
    return lda,cs_list,logper_list                            #----------------这里也改了

In [ ]:
text=pd.read_csv("./huanqiu_news_with_title_2.csv")
#看text里有没有空值的项，因为split会发错误
drop_na=[]
for i in range(text.shape[0]):
    if pd.isnull(text.loc[i]).any():
        drop_na.append(i)

text=text.drop(drop_na)
text=text.reset_index(drop=True)

text.head()

In [ ]:
c=[i.split(" ") for i in text["微博正文(有标题切词后)"]]

In [ ]:
tn=50 #把确定好的主题数放进这里（和上一小节中的一致）
ga=0.5#把确定好的gamma放进这里

a,b=tw_lda_get_tfidf(text,gamma=ga)
lda3,cs3,logper3=cs_bar_10(c,
                           tn,
                           "tw_lda",
                           corpus_tfidf_tw_lda=a,
                           dictionary_tw_lda=b)
text_topics3=lda3.get_document_topics(a)

TW-LDA可视化

In [ ]:
import pyLDAvis
import pyLDAvis.gensim


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda3, a, b)

end_lda = time.clock()

#pyLDAvis.show(vis)

In [ ]:
pyLDAvis.show(vis)

分主题看微博量

In [ ]:
lda3.print_topics(-1,20)

In [ ]:
text_topics_dict3={}
for i in range(tn):
    text_topics_dict3[i]=[]
    
text_topics_no_dict3={}
for i in range(tn):
    text_topics_no_dict3[i]=[]

for i in range(len(text_topics3)):
    dict_text_topics3=dict(text_topics3[i])
    #print(dict_text_topics3)
    for key,value in dict_text_topics3.items():
        if(value == max(dict_text_topics3.values()) and str(max(dict_text_topics3.values()))!="0.02"):
            text_topics_dict3[key].append(value)
            text_topics_no_dict3[key].append(i)

In [ ]:
lda3.print_topics(-1,20)

In [ ]:
lda3.show_topic(48,    #改变这个看不同主题,总结规律
                      #（和上一小节好像差不多，看看是不是）
                20)   #20个关键词

In [ ]:
#编号为48的主题
topic_id=48
print(lda3.show_topic(topic_id,20))
import re
time_3={}
for i in range(1,13):
    month_text=0
    if len(str(i))==1:
        mon="0"+str(i)
    else:
        mon=str(i)
    month_re='2019-'+mon
    time_list=text["发布时间"][text_topics_no_dict3[topic_id]]
    for j in time_list:
        if month_re in j:
            month_text=month_text+1
    time_3[mon]=month_text
print(time_3)

import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

plt.figure(figsize=(13,5),dpi=50)

month_re_list=[]
for i in range(1,13):
    if len(str(i))==1:
        mon="0"+str(i)
    else:
        mon=str(i)
    month_re='2019-'+mon
    month_re_list.append(month_re)
month_re_list.append('2020-01')
#print("month_re_list",month_re_list)
new_ticks =month_re_list

plt.xticks(range(1,14),new_ticks)

plt.ylim(0, 250)

x = [int(i[0]) for i in time_3.items()]
x.append(13)

y = [int(i[1]) for i in time_3.items()]
y.append(0)
#np.linspace(0, 10, num=11, endpoint=True)

f2 = interp1d(x, y, kind='zero')

xnew = np.linspace(1, 13, num=1001, endpoint=False)
#plt.plot(x, y, 'o')
plt.plot(xnew, f2(xnew), '-',color='blue')
plt.legend(['Topic1', 'zero'], loc='best')

x1=[int(i[0]) for i in time_3.items()]
y1=[int(i[1]) for i in time_3.items()]

for m,n in zip(x1,y1):
    plt.text(m+0.5, n+0.15, '%.0f' % n, ha='right', va= 'bottom',fontsize=11)

plt.show()
